In [ ]:
def getSecret(secretName):
    linked_service = "ifmpmvault"
    akv_name = "ifm-vault"

    # Fetch the key from Azure Key Vault
    secretValue = mssparkutils.credentials.getSecret(
        linkedService=linked_service,
        akvName=akv_name, 
        secret=secretName)
    return secretValue

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset

from azureml.train.automl.run import AutoMLRun
from azureml.train.automl import AutoMLConfig
from azureml.automl.runtime.onnx_convert import OnnxConverter
from azureml.core.model import Model
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd
import joblib

from azureml.core.model import Model

subscription_id = getSecret('subscription-id') 
resource_group = getSecret('resource-group') 
workspace_name = getSecret('workspace-name') 

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

# pull all metrics of best run
experiment_name = 'PredictiveMaintenanceExperiment'

for automl_run in ws.experiments[experiment_name].get_runs():
    best_run, fitted_model = automl_run.get_output()  # We are taking the first run. You can update this if you like to take a different run
    break

#save the model to a local file
model_path = 'failure_prediction_model'
joblib.dump(fitted_model, model_path)

model_name = "failure_prediction_model"
registered_model = Model.register(model_path = model_path, # this points to a local file
                       model_name = model_name, # name the model is registered as
                       tags = {'type': "classification"}, 
                       description = "Failure Prediction model", 
                       workspace = ws)